In [ ]:
# %load ./init.ipy
%reload_ext autoreload
%autoreload 2

# Builtin packages
from datetime import datetime
from importlib import reload
import logging
import os
from pathlib import Path
import sys
import warnings

# standard secondary packages
import astropy as ap
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.stats
import tqdm.notebook as tqdm

# development packages
import kalepy as kale
import kalepy.utils
import kalepy.plot

# --- Holodeck ----
import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR, SPLC, NWTG
import holodeck.gravwaves
import holodeck.evolution
import holodeck.population

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

# Load log and set logging level
log = holo.log
log.setLevel(logging.INFO)

In [ ]:
DEF_NUM_FBINS = 40
DEF_PTA_DUR = 16.03     # [yrs]

# Choose observed GW-Frequency bins based on nyquist sampling
fobs_edges = utils.nyquist_freqs_edges(DEF_PTA_DUR*YR, cad=0.1*YR)
fobs_edges = fobs_edges[:DEF_NUM_FBINS+1]
fobs = utils.midpoints(fobs_edges)

## Basic Example

In [ ]:
gsmf = holo.sam.GSMF_Schechter()               # Galaxy Stellar-Mass Function (GSMF)
gpf = holo.sam.GPF_Power_Law()                 # Galaxy Pair Fraction         (GPF)
gmt = holo.sam.GMT_Power_Law()                 # Galaxy Merger Time           (GMT)
mmbulge = holo.relations.MMBulge_Standard()    # M-MBulge Relation            (MMB)

sam = holo.sam.Semi_Analytic_Model(
    gsmf=gsmf, gpf=gpf, gmt=gmt, mmbulge=mmbulge, shape=40,    
    ZERO_GMT_STALLED_SYSTEMS=True,
)

In [ ]:
hard_gw = holo.hardening.Hard_GW()
gwb = sam.gwb(
    fobs, realize=20, hard=hard_gw,
    zero_coalesced=True, zero_stalled=False, use_redz_after_hard=False
)

In [ ]:
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', ylabel='Characteristic Strain $h_c$')

xx = (fobs[1:] + fobs[:-1]) * 0.5
ax.plot(xx, np.median(gwb, axis=1))

ax.fill_between(xx, *np.percentile(gwb, [25, 75], axis=1), alpha=0.2)

plt.show()

In [ ]:
Fixed_Time = holo.hardening.Fixed_Time
fixed = Fixed_Time.from_sam(sam, 2 * GYR)

In [ ]:
gwb_fixed = sam.gwb(fobs, realize=3, hard=fixed)    # calculate many different realizations

In [ ]:
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', ylabel='Characteristic Strain $h_c$')

xx = (fobs[1:] + fobs[:-1]) * 0.5
ax.plot(xx, np.median(gwb_fixed, axis=1))

ax.fill_between(xx, *np.percentile(gwb_fixed, [25, 75], axis=1), alpha=0.2)

plt.show()

## Compare GW-Only to Phenomenological Hardening Model

In [ ]:
sam = holo.sam.Semi_Analytic_Model(shape=10)

In [ ]:
gwb_gw = sam.gwb(fobs_edges, realize=300, hard=holo.hardening.Hard_GW(),
                 zero_stalled=False, use_redz_after_hard=False)

In [ ]:
NREALS = 100
# hard_times = np.linspace(0.1, 12.0, 3)
hard_times = [0.1, 0.5, 1.0, 2.0, 5.0, 10.0]
ntimes = len(hard_times)
nfobs = fobs.size
gwb_ph = np.zeros((ntimes, nfobs, NREALS))

for ii, htime in enumerate(hard_times):
    print(ii, htime)
    hard = holo.hardening.Fixed_Time.from_sam(sam, htime*GYR)
    _gwb = sam.gwb(fobs_edges, realize=NREALS, hard=hard, zero_stalled=True)
    gwb_ph[ii, :, :] = _gwb


In [ ]:
PLAW_FIT_NFREQS = 0
COLORBAR = False

if COLORBAR:
    kw = dict(bottom=0.25)
else:
    kw = {}
    
fig, axes = plot.figax(figsize=[10, 6], ncols=2, wspace=0.1, **kw)

xx = fobs * YR
NSAMP = 0

axes[0].set(xlabel=plot.LABEL_GW_FREQUENCY_YR, ylabel=plot.LABEL_CHARACTERISTIC_STRAIN)
axes[1].set(xlabel=plot.LABEL_GW_FREQUENCY_YR, ylabel="Ratio (phenom / GW)")

smap = plot.smap(hard_times, log=True, cmap='Spectral')
colors = smap.to_rgba(hard_times)

med_gw = np.median(gwb_gw, axis=-1)
# log10amp, gamma = utils.fit_powerlaw(xx[:PLAW_FIT_NFREQS], med_gw[:PLAW_FIT_NFREQS])
gamma = -2.0 / 3.0
if PLAW_FIT_NFREQS > 1:
    log10amp = utils.fit_powerlaw_fixed_index(xx[:PLAW_FIT_NFREQS], med_gw[:PLAW_FIT_NFREQS], index=gamma)
else:
    amp = med_gw[0] * np.power(xx[0], +2.0/3.0)
    log10amp = np.log10(amp)

plaw = np.power(10.0, log10amp + gamma * np.log10(xx))
ref = med_gw

# plot.draw_gwb(axes[0], xx, gwb_gw, color='k', nsamp=NSAMP)
axes[0].plot(xx, med_gw, color='k', zorder=10)
axes[0].plot(xx, plaw, ls=':', color='k', alpha=0.5, lw=2.0)
axes[1].plot(xx, np.ones_like(ref), ls='-', color='k', alpha=0.5)

labels = []
handles = []
for ii, htime in enumerate(hard_times):
    lab = f"{htime:.1f}"
    col = colors[ii]
    hh = plot.draw_gwb(axes[0], xx, gwb_ph[ii, :, :], color=col, nsamp=NSAMP, fracs=[0.5])
    med = np.median(gwb_ph[ii], axis=-1)
    axes[1].plot(xx, med/ref, color=col, alpha=0.5)
    handles.append(hh)
    labels.append(lab)

if COLORBAR:
    cax = fig.add_axes([0.2, 0.08, 0.6, 0.03])
    plt.colorbar(smap, label='Lifetime [Gyr]', orientation='horizontal', cax=cax)
else:
    axes[0].legend(handles, labels, fontsize=8)    

plot._twin_hz(axes[0])
plot._twin_hz(axes[1])

axes[1].yaxis.set_label_position('right')
axes[1].yaxis.set_ticks_position('right')

plt.show()
filekey = ""
fname = f"phenom-hard{filekey}.png"
fname = Path(holo._PATH_OUTPUT).joinpath(fname)
fig.savefig(fname, dpi=400)
print(f"Saved to {fname}, size {utils.get_file_size(fname)}")


In [ ]:
sepa = np.logspace(-6, 4, 30)[::-1] * PC
ntimes = len(hard_times)
numbers = []
taus = []

for ii, htime in enumerate(tqdm.tqdm_notebook(hard_times)):
    print(ii, htime, "Gyr")
    hard = holo.hardening.Fixed_Time.from_sam(sam, htime*GYR)
    edges, number, details = sam.dynamic_binary_number(hard, sepa=sepa, return_details=True)
    numbers.append(number)
    taus.append(details['tau'])
    break

In [ ]:
xx = sepa / PC
# hc2 = holo.gravwaves.char_strain_sq_from_bin_edges(edges)
mc = holo.utils.chirp_mass_mtmr(edges[0][:, np.newaxis], edges[1][np.newaxis, :])
dc = cosmo.comoving_distance(edges[2]).cgs.value

print(mc.shape)

tau = taus[0]
mc = mc[:, :, np.newaxis, np.newaxis] * np.ones_like(tau)
hc2 = np.power(mc, 4.0/3.0) / dc[np.newaxis, np.newaxis, :, np.newaxis]**2

nn = numbers[0]
nn = np.moveaxis(nn, -1, 0).reshape(xx.size, -1)
tau = np.moveaxis(tau, -1, 0).reshape(xx.size, -1)

fig, ax = plot.figax(ylim=[1e1, 1e9])
plot.draw_med_conf(ax, xx, tau/YR)
plot.draw_med_conf(ax, xx, tau/YR, weights=nn)

weights = np.moveaxis(hc2, -1, 0).reshape(xx.size, -1)
plot.draw_med_conf(ax, xx, tau/YR, weights=weights)

tw = ax.twinx()
tw.set(yscale='log')
nn = utils._integrate_grid_differential_number(edges, numbers[0], freq=False)
tw.plot(xx, np.sum(np.nan_to_num(nn), axis=(0, 1, 2)), 'k--')
# tw.plot(xx, np.sum(numbers[0], axis=(0, 1, 2)))

plt.show()

In [ ]:
xx = sepa / PC
plt.loglog(xx, number.sum(axis=(0, 1, 2)))
plt.show()

# Vary hardening-rate power-law parameters

In [ ]:
pta_dur = 16.03 * YR
nfreqs = 20
hifr = nfreqs/pta_dur
pta_cad = 1.0 / (2 * hifr)
fobs_cents = holo.utils.nyquist_freqs(pta_dur, pta_cad)
fobs_edges = holo.utils.nyquist_freqs_edges(pta_dur, pta_cad)


## Inner Power-Law

In [ ]:
hard._gamma_df, hard._gamma_sc

In [ ]:
space = holodeck.param_spaces.PS_Uniform_06(log, 100, 30, None)
nreals = 20

# new_pars = dict(hard_gamma_inner=0.0, hard_gamma_outer=+1.0)
inner_list = [-1.5, -1.0, -0.5, 0.0]
sepa = np.logspace(-3, 3, 40)[::-1] * PC

sams = []
gwbs = []
dets = []

for inner in tqdm.tqdm(inner_list):
    new_pars = dict(hard_gamma_inner=inner)
    sam, hard = space.model_for_normalized_params(
        0.5, sam_shape=space.sam_shape, new_def_params=new_pars,
    )
    _gwb = sam.gwb(fobs_edges, hard=hard, realize=nreals,)
    gwbs.append(_gwb)

    edges, dnum, _details = sam.dynamic_binary_number(hard, sepa=sepa, return_details=True)
    dets.append(_details)
    # break

In [ ]:
fig, ax = plot.figax()

labels = []
handles = []
for ii, gwb in enumerate(gwbs):
    hh = plot.draw_gwb(ax, fobs_cents*YR, gwb)
    labels.append(inner_list[ii])
    handles.append(hh)
    
ax.legend(handles, labels)
plt.show()

In [ ]:
import zcode.math as zmath
import zcode.plot as zplot

mup = 1e8
midx = zmath.argnearest(sam.mtot/MSOL, mup)
print(mup, midx, sam.mtot[midx]/MSOL)
qup = 0.2
qidx = zmath.argnearest(sam.mrat, qup)
print(qup, qidx, sam.mrat[qidx])

fig, ax = plot.figax()
xx = sepa/PC

labels = []
handles = []
for ii, det in enumerate(dets):
    # vals = det['tau']/GYR
    vals = -det['dadt']*YR/PC
    # print(vals.shape)

    # vals = np.mean(vals[midx:, qidx:, :, :], axis=(0, 1, 2))
    vals = vals[midx:, qidx:, :, :].reshape(-1, sepa.size)
    print(vals.shape)
    plot.draw_med_conf(ax, xx, vals.T, fracs=[0.5])

    # labels.append(inner_list[ii])
    # handles.append(hh)

for pl in np.arange(1.0, 3.1, 0.5):
    l1, = ax.plot(xx, np.power(xx, pl), color='0.5', ls='--', alpha=0.25)    
    l2, = ax.plot(xx, np.power(xx, -pl), color='0.5', ls='--', alpha=0.25)    
    zplot.label_line(ax, l1, f"${pl:+.1f}$", x=0.1, flip_rotation=True, alpha=0.35)
    zplot.label_line(ax, l2, f"${-pl:+.1f}$", x=0.9, flip_rotation=True, alpha=0.35)
    
ax.legend(handles, labels)
plt.show()

In [ ]:
new_sams = []
new_gwbs = []

sam_shape = 80
nreals = 100

params = {}
for ii, pname in enumerate(space.param_names):
    ss = space._dists[ii](0.5)    # convert to actual parameter values
    params[pname] = ss           # store to dictionary

print(params)

# scatter_values = [0.2, 0.4, 0.6, 0.8, 1.0, 1.2]
scatter_values = [0.3, 0.6, 0.9, 1.2]
for scatter in scatter_values:
    
    # params['mmb_mamp_log10'] = 9.5
    params['mmb_mamp_log10'] = 11
    params['mmb_scatter_dex'] = scatter
    print(params)

    # new_pars = dict(hard_gamma_inner=0.0, hard_gamma_outer=+1.0)
    def_pars = dict()

    sam, hard = space.model_for_params(
        params, sam_shape=sam_shape, new_def_params=def_pars
    )
    new_sams.append(sam)

    gwb = sam.gwb(fobs_edges, realize=nreals, hard=hard)
    new_gwbs.append(gwb)

    dn = sam._dynamic_binary_number[:, :, :, 0]
    print(f"{sam.shape=} {dn.sum()=:.8e}")

In [ ]:
fig, axes = plt.subplots(ncols=2)

med = []
std = []
for gwb in gwbs:
    med.append(np.median(gwb[0]))
    std.append(np.std(gwb[0]))
    
axes[0].plot(scatter_values, med)
axes[1].plot(scatter_values, std)

for ax in axes:
    ax.set(yscale='log')
    
plt.show()

In [ ]:
fig, ax = holo.plot.figax()

for gwb in gwbs:
    holo.plot.draw_gwb(ax, fobs_cents*YR, gwb, nsamp=0)

plt.show()

In [ ]:
fig, ax = holo.plot.figax()

for gwb in gwbs:
    holo.plot.draw_gwb(ax, fobs_cents*YR, gwb, nsamp=0)

plt.show()